In [26]:
strategy = {
    "id": 33,
    "title": "Why don't we Just?",
    "type": "in-play",
    "outcomes": [
      "home_win"
    ],
    "is_public": 1,
    "timer": {
      "to": 70,
      "from": 0,
      "time": "atMinuteX",
      "minute": 75
    },
    "hit_rate": "0.00",
    "fixtures_found": 0,
    "last_checked": None,
    "trusted": 0,
    "active": 1,
    "leagues": [
      1117,
      1515,
      1103,
      154,
      157,
      160,
      163,
      166,
      169,
      94,
      97,
    ],
    "note": "",
    "user_id": 1,
    "created_at": "2021-09-22T03:24:17.000Z",
    "updated_at": "2021-09-22T03:24:17.000Z",
    "strategy_prematch_rules": [
      {
        "code": "played",
        "category": "general",
        "overall": "played_overall",
        "home": "played_home",
        "away": "played_away",
        "id": 24,
        "strategy_id": 33,
        "uid": "played_general",
        "location": "overall",
        "team": "both",
        "comparator": "=",
        "value": "1.00",
        "values": [
          15,
          60
        ]
      }
    ],
    "strategy_inplay_rules": [
      {
        "first_code": "goals",
        "second_code": "goals",
        "first_category": "statistics",
        "second_category": "statistics",
        "comparator": "=",
        "value": 1,
        "first_team": "home",
        "second_team": "away"
      },
       {
        "first_code": "corners",
        "second_code": "corners",
        "first_category": "statistics",
        "second_category": "numeric",
        "comparator": "=",
        "value": 8,
        "first_team": "either_team",
        "second_team": "away"
    },
      {
        "first_code": "ft_result",
        "second_code": "goals",
        "first_category": "odds",
        "second_category": "numeric",
        "comparator": "=",
        "value": 23,
        "first_team": "sum_of_teams",
        "second_team": "away"
      },
      {
        "first_code": "ft_result",
        "second_code": "goals",
        "first_category": "odds",
        "second_category": "odds",
        "comparator": "=",
        "value": 23,
        "first_team": "winning_team",
        "second_team": "losing_team"
      }
    ]
  }

In [27]:
def __parse_time(timer):
    from_minute = timer["from"]
    to_minute = timer["to"]
    time = timer["time"]
    minute = timer["minute"]

    time_map = {
        "disabled": "minute != NULL",
        "atMinuteX": f"minute = {minute}",
        "xMinutesAgo": f"minute = minute - {minute}",
        "betweenMinutesXandY": f"minute BETWEEN {from_minute} to {to_minute}",
        "pastXminutes": f"minute > minute - {minute}",
        "sinceMinuteX": f"minute >= {minute}",
        "untilMinuteX": f"minute <= {minute}",
        "during1stHalf": "minute <= 45",
        "during2ndHalf": "minute > 45",
        "atHalfTime": "minute = 45",
        "atFullTime": "minute = 90"
    }
    return time_map[time]


In [28]:
def __has_team(code, category):
    if category == "odds":
        if code in {"dnb", "ht_result", "ft_result"}:
            return True
    else:
        return True
    return False


def __parse_single_rule(code, category, team):
    if category == "odds":
        table_name = "liveodds"
    else:
        table_name = "stats"
    if category == "statistics":
        if team == "sum_of_teams":
            return f"{table_name}.home.{code} + {table_name}.away.{code}"
        elif team == "either_team":
            return [f"{table_name}.home.{code}", f"{table_name}.away.{code}"]
        else:
            return f"{table_name}.{team}.{code}"
    elif __has_team(code, category):
        if team == "sum_of_teams":
            return f"{table_name}.{code}_home + {table_name}.{code}_away"
        elif team == "either_team":
            return [f"{table_name}.{code}_home", f"{table_name}.{code}_away"]
        else:
            return f"{table_name}.{code}_{team}"
    else:
        return f"{table_name}.{code}"

# def get_table_name()

def __parse_rule(first_code, second_code, first_category, second_category, comparator, value, first_team, second_team):
    first_field = __parse_single_rule(code=first_code, category=first_category, team=first_team)
    if second_category == "numeric":
      second_field = value
    else:
      second_field = __parse_single_rule(code=second_code, category=second_category, team=second_team)
    if not isinstance(first_field, list):
        return f"{first_field} {comparator} {second_field}"
    else:
        return " AND ".join(map(lambda field: f"{field} {comparator} {second_field}", first_field))


In [29]:
in_play_conditions = []
for rule in strategy["strategy_inplay_rules"]:
  first_code = rule["first_code"]
  second_code = rule["second_code"]
  first_category = rule["first_category"]
  second_category = rule["second_category"]
  comparator = rule["comparator"]
  value = rule["value"]
  first_team = rule["first_team"]
  second_team  = rule["second_team"]
  condition = __parse_rule(first_code, second_code, first_category, second_category, comparator, value, first_team, second_team)
  in_play_conditions.append(condition)

In [30]:
in_play_sql = "\nAND\n".join(in_play_conditions)

In [31]:
included_leagues = ",".join(map(str, strategy["leagues"]))
included_leagues = f"({included_leagues})"

In [32]:
timer_sql = __parse_time(strategy["timer"])

In [33]:
def __condition_for_probability(rule):
  return f'probability.{rule["code"]} BETWEEN {rule["values"][0]} AND {rule["values"][1]}'

def __condition_for_odds(rule):
  return f'odds.{rule["code"]} {rule["comparator"]} {rule["value"]} AND odds.type = "preodds"'

def __condition_for_team(rule, team):
  return f'{team}.{rule[rule["location"]]} BETWEEN {rule["values"][0]} AND {rule["values"][1]}'

def __condition_for_aggregate_stats(rule):
  if rule["team"] == "home":
    condition = __condition_for_team(rule, "home")
  elif rule["team"] == "away":
    condition =  __condition_for_team(rule, "away")
  else:
    condition =__condition_for_team(rule, "home") + " AND " + __condition_for_team(rule, "away")
  return condition


pre_match_conditions = []
for rule in strategy["strategy_prematch_rules"]:
  if rule["category"] == "probability":
    condition = __condition_for_probability(rule)
  elif rule["category"] == "odds":
    condition = __condition_for_odds(rule)
  else:
    condition = __condition_for_aggregate_stats(rule)
  pre_match_conditions.append(condition)

In [34]:
pre_match_sql = "\nAND\n".join(pre_match_conditions)

In [35]:
pre_match_sql

'home.played_overall BETWEEN 15 AND 60 AND away.played_overall BETWEEN 15 AND 60'

In [36]:
sql = f"""
SELECT * FROM fixtures 
WHERE 
{timer_sql}
AND
STATUS = LIVE
AND
league_id in {included_leagues}"""

if in_play_sql:
  sql+= f"\nAND\n{in_play_sql}"
if pre_match_sql:
  sql+= f"\nAND\n{pre_match_sql}"

print(sql)


SELECT * FROM fixtures 
WHERE 
minute = 75
AND
STATUS = LIVE
AND
league_id in (1117,1515,1103,154,157,160,163,166,169,94,97)
AND
stats.home.goals = stats.away.goals
AND
stats.home.corners = 8 AND stats.away.corners = 8
AND
liveodds.home_win + liveodds.away_win = 23
AND
liveodds.ft_result_winning_team = liveodds.goals
AND
home.played_overall BETWEEN 15 AND 60 AND away.played_overall BETWEEN 15 AND 60
